In [12]:
from bs4 import BeautifulSoup
from bs4 import NavigableString, Tag

import requests
import json
import os

def getInfoDicFromJavhoo(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("img", class_="alignnone size-full")['src']
    fullTitle = bsSoup.find("h1", class_="h3-size entry-title").get_text()
    title = fullTitle[fullTitle.find(' ')+1:] 
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", class_="project_info")  
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :')
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                else:
                    body = ""
                valueDic[header] = body
                p = p.find_next_sibling('p')
        else:
            header = p.get_text().strip(' :')
            body = []
            p = p.find_next_sibling('p')
            if (p.contents[0]['class'] == ['genre']):
                 for child in p.contents[0]:
                    body.append(child.get_text())
            p = p.find_next_sibling('p')
            valueDic[header] = body

    valueDic['網站'] = 'Javhoo'
    return valueDic


def getDicFromJavhoo(avID):
    url = "https://www.javhoo.com/av/" + avID.lower()
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavhoo(soup)



def getInfoDicFromJavBus(bsSoup):
    valueDic ={}

    coverPath = bsSoup.find("a", {'class': "bigImage"}).find('img')['src']
    fullTitle = bsSoup.find("div", {'class': "container"}).find('h3').get_text()
    title = fullTitle[fullTitle.find(' ')+1:]  # title might contains actress name, need to remove it later
    ID = fullTitle[0:fullTitle.find(' ')]
    valueDic['封面圖'] = coverPath
    valueDic['標題'] = title
    valueDic['序號'] = ID
    
    pInfo = bsSoup.find("div", {'class': "col-md-3 info"})   
    p = pInfo.find('p')

    while p:
        if not p.has_attr('class'):      # most p does not have class
            if p.contents[0].has_attr('class') and p.contents[0]['class'] == ['header']:
                header = p.contents[0].get_text().strip(' :\n')
#                print('header is: ', header)
                if (len(p) == 2):
                    if isinstance(p.contents[1], NavigableString):
                        body = p.contents[1]
                    else:
                        body = p.contents[1].get_text()
                elif (len(p) == 3):
                    body = p.contents[2].get_text() # try&error, the last element contains data
                else:
                    body = ""
#                print('body is: ', body)
                valueDic[header] = body.strip(' :\n')
                p = p.find_next_sibling('p')
        elif p.has_attr('class') and p['class'] == ['header']:
            header = p.get_text().strip(' :\n')
#            print('header2 is: ', header)
            body = [] 
            p = p.find_next_sibling('p')
            for item in p:
                if not isinstance(item, NavigableString) and item.has_attr('class') and item['class'] == ['genre']:
                    body.append(item.get_text().strip(' :\n'))           
#            print('body2 is: ', body)
            valueDic[header] = body
            p = p.find_next_sibling('p')            
        elif p.has_attr('class') and p['class'] == ['star-show']:
            header = p.get_text().strip(' :\n')
#            print('header3 is: ', header)
            body = [] 
            p = p.find_next_sibling('p')
            for item in p:            
                if not isinstance(item, NavigableString) and item.has_attr('class') and item['class'] == ['genre']:
                    body.append(item.get_text().strip(' :\n'))
            p = p.find_next_sibling('p')  
#            print('body3 is: ', body)
            valueDic[header] = body

    valueDic['網站'] = 'JavBus'
    return valueDic


def getDicFromJavBus(avID):
    url = "https://www.javbus.com/" + avID.upper()
    r = requests.get(url)
    if r.status_code == 404:
        return {}
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return getInfoDicFromJavBus(soup)






def downloadImage(webPath, avID):
    ext = webPath[webPath.rfind('.'):]
    filename = localFolder + '/' + avID + ext
    r = requests.get(webPath, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(1024):
                f.write(chunk)  

def dumpDic2Json(valueDic):
    ID = valueDic['序號']
    filename = localFolder + '/' + ID +'.json'
    f = open(filename, "w")
    json.dump(valueDic, f)
    f.close()
    
def loadDicFromJson(avID):
    filename = localFolder + '/' + avID +'.json'    
    f = open(filename, 'r')
    valueDic = json.load(f)
    f.close()
    return valueDic
    
def checkAvIdExist(avID):
    filename = localFolder + '/' + avID +'.json'
    if os.path.exists(filename):
        return True
    return False

def checkAvIdImgExist(avID):
    jpg = localFolder + '/' + avID +'.jpg'
    png = localFolder + '/' + avID +'.png'
    if os.path.exists(jpg) or os.path.exists(png):
        return True
    return False


def completeAvID(avID, showInfo = False):
    vDic = {}
    #AvId status, 1 not download before, 2 not exist on Javhoo, 3 downloaded
    if not checkAvIdExist(avID):
        vDic = getDicFromJavhoo(avID)
        if (len(vDic) == 0):
            vDic = getDicFromJavBus(avID)
            if (len(vDic) == 0):
                return False;
        if vDic['ID'].lower() != avID.lower():
            raise ValueError('ID got and given not equal', vDic['ID'].lower(), avID.lower())
        dumpDic2Json(vDic)
        if showInfo:
            print("Downloaded ", avID)
    else:
        vDic = loadDicFromJson(avID)

    # check if img exist, otherwise download img
    if not checkAvIdImgExist(avID):
        downloadImage(vDic['封面圖'], avID)
        if showInfo:
            print("Img Downloaded", avID)
            
    return True


def dumpResult2Json(avIDResult):
    filename = 'avIDResult.json'
    f = open(filename, "w")
    json.dump(avIDResult, f)
    f.close()
    
def loadResultFromJson():
    filename = 'avIDResult.json'    
    f = open(filename, 'r')
    avIDResult = json.load(f)
    f.close()
    return avIDResult

def countAVResult(avIDResult):
    count0 = 0
    count404 = 0
    count200 = 0
    for key, value in avIDResult.items():
        if value[0] == 0:
            count0+=1
        elif value[0] == 200:
            count200+=1
        elif value[0] == 404:
            count404+=1
    print("AVIDResult: 0~", count0, " 200~", count200, " 404~", count404)

from itertools import islice
from random import randint
import time
import random

def randomGet(avIDResult, count):
    try_batch = list(islice(avIDResult.items(), len(avIDResult)))
    c_200 = 0
    c_404 = 0
    c_999 = 0

    x = [randint(0, len(avIDResult)) for p in range(0, count)]

    for i in x:
        item = try_batch[i]
        t = 0
        if item[1][0] == 0:    # not get from web before
            result = completeAvID(item[0])
            if result:
                item[1][0] = 200
                c_200+=1
                print("---- {0: <10} downloaded,    ".format(item[0]), end='')
                t = random.uniform(0.5, 5.0)             
            else:
                item[1][0] = 404
                c_404+=1
                print("@@@@ {0: <10} not on Javhoo, ".format(item[0]), end='')
                t = random.uniform(0.5, 1.0)                   
        else:
            print("!!!! {0: <10} already there, ".format(item[0]), end='')
            c_999+=1
        print("pause {0:.2f} seconds...".format(t))
        time.sleep(t)
    print("$$$$$ Batch for {0} items, {1} [200], {2} [404], {3} [already].".format(count, c_200, c_404, c_999))
        
    
localFolder = '/Users/yiwenliu/Desktop/Projects/AV'

In [3]:
avIDResult = loadResultFromJson()

In [ ]:
start_time = time.time()
for i in range(5):
    randomGet(avIDResult, 10)
    dumpResult2Json(avIDResult)
countAVResult(avIDResult)
elapsed_time = time.time() - start_time
print("Time consumed: {0:.2f} secs".format(elapsed_time))

In [4]:
countAVResult(avIDResult)

AVIDResult: 0~ 4989  200~ 711  404~ 232


In [5]:
randomGet(avIDResult, 2)

@@@@ XV-700     not on Javhoo, pause 0.98 seconds...
---- CETD-230   downloaded,    pause 1.57 seconds...
$$$$$ Batch for 2 items, 1 [200], 1 [404], 0 [already].


In [7]:
vDic = loadDicFromJson("CETD-230")

In [8]:
vDic

{'封面圖': 'https://pics.javhoo.net/cover/en/4kdv_b.jpg',
 '導演': '匠野透',
 '序號': 'CETD-230',
 '標題': '私痴漢バスに乗ります。3 ワイセツな指先が清楚な若妻の理性を崩す…夫同伴のバス車内で痴漢に犯される人妻 愛乃ゆな',
 '演員': ['愛乃ゆな'],
 '番號': 'cetd-230\xa0/\xa0有碼',
 '發行商': 'セレブの友',
 '發行日期': '2014-12-13',
 '系列': '私痴漢バスに乗ります。',
 '製作商': 'セレブの友',
 '長度': '124分鐘',
 '類別': ['DMM獨家', '單體作品', '已婚婦女', '性騷擾', '戲劇', '新娘，年輕妻子']}

In [9]:
completeAvID("ABP-612")

True

In [10]:
vDic = loadDicFromJson("ABP-612")

In [11]:
vDic

{'封面圖': 'https://pics.javhoo.net/2017/07/cover/ABP-612.jpg',
 '導演': 'マンハッタン木村',
 '序號': 'ABP-612',
 '標題': '人生初・トランス状態 激イキ絶頂セックス 39 愛音まりあ',
 '演員': ['愛音まりあ'],
 '番號': 'abp-612\xa0/\xa0有碼',
 '發行商': 'ABSOLUTELY PERFECT',
 '發行日期': '2017-07-07',
 '系列': '人生初・トランス狀態 激イキ絶頂セックス',
 '製作商': 'プレステージ',
 '長度': '174分鐘',
 '類別': ['口交', '單體作品', '子宮頸', '無毛', '玩具', '高畫質']}